## Import all files

In [1]:
import os
import zipfile
import pandas as pd

In [21]:
# Path to the folder containing zip files
folder_path = 'Satellite data'

In [22]:
# Dictionary to store the DataFrames
dataframes = {}

### Test one file

In [24]:
import zipfile
import pandas as pd
import os

# Path to the folder containing zip files
folder_path = 'Satellite data'

# Specific zip file and TQ file to read
zip_filename = 'GLORIA_SatelliteAccounts_059_1990.zip'
specific_tq_file_name = '20231117_120secMother_AllCountries_002_TQ-Results_1990_059_Markup001(full).csv'

# Path to the specific zip file
zip_path = os.path.join(folder_path, zip_filename)

# Dictionary to store the DataFrame
dataframes = {}

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Check if the specific TQ file is in the zip
    if specific_tq_file_name in zip_ref.namelist():
        # Read the CSV file into a DataFrame
        with zip_ref.open(specific_tq_file_name) as file:
            df = pd.read_csv(file)
            # Store the DataFrame in the dictionary with the filename as the key
            dataframes[specific_tq_file_name] = df
            print(f"Loaded {specific_tq_file_name} into DataFrame.")



Loaded 20231117_120secMother_AllCountries_002_TQ-Results_1990_059_Markup001(full).csv into DataFrame.


In [28]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5981 entries, 0 to 5980
Columns: 39360 entries, 0 to 0.39257
dtypes: float64(19556), int64(19804)
memory usage: 1.8 GB


In [29]:
# Print a concise summary of the DataFrame
print("\nSummary of the DataFrame:")
print(df.describe(include='all'))


Summary of the DataFrame:
                 0          0.1            0.2           0.3       209553  \
count  5981.000000  5981.000000    5981.000000   5981.000000  5981.000000   
mean      1.922038     4.425283      40.399570      6.470695     2.131069   
std      69.419922   167.713523    2620.481021    218.734446    69.213205   
min       0.000000     0.000000       0.000000      0.000000     0.000000   
25%       0.000000     0.000000       0.000000      0.000000     0.000000   
50%       0.000000     0.000000       0.000000      0.000000     0.000000   
75%       0.000000     0.000000       0.000000      0.000000     0.000000   
max    3838.633204  9453.000000  201170.880000  10822.609530  3428.109011   

                0.4           0.5          0.6           0.7          0.8  \
count   5981.000000  5.981000e+03  5981.000000   5981.000000  5981.000000   
mean      50.606025  5.250928e+02     0.678763      5.415432     0.333900   
std     1184.766129  4.056604e+04    26.944835  

In [30]:
# Print data types of each column
print("\nData Types of Each Column:")
print(df.dtypes)


Data Types of Each Column:
0          float64
0.1        float64
0.2        float64
0.3        float64
209553     float64
            ...   
0.39253      int64
0.39254      int64
0.39255      int64
0.39256      int64
0.39257      int64
Length: 39360, dtype: object


In [32]:
# Check for missing values in the DataFrame
missing_values = df.isnull().sum()

# Print a message indicating if there are missing values
if missing_values.sum() > 0:
    print("\nMissing Values Found:")
    print(missing_values[missing_values > 0])
else:
    print("\nNo Missing Values Found.")



No Missing Values Found.


In [33]:
# Print the first few rows of the DataFrame
print("\nFirst Few Rows:")
print(df.head())


First Few Rows:
     0  0.1        0.2  0.3  209553  0.4  0.5  0.6  0.7  0.8  ...  0.39248  \
0  0.0  0.0  201170.88  0.0     0.0  0.0  0.0  0.0  0.0  0.0  ...        0   
1  0.0  0.0       0.00  0.0     0.0  0.0  0.0  0.0  0.0  0.0  ...        0   
2  0.0  0.0       0.00  0.0     0.0  0.0  0.0  0.0  0.0  0.0  ...        0   
3  0.0  0.0       0.00  0.0     0.0  0.0  0.0  0.0  0.0  0.0  ...        0   
4  0.0  0.0       0.00  0.0     0.0  0.0  0.0  0.0  0.0  0.0  ...        0   

   0.39249  0.39250  0.39251  0.39252  0.39253  0.39254  0.39255  0.39256  \
0        0        0        0        0        0        0        0        0   
1        0        0        0        0        0        0        0        0   
2        0        0        0        0        0        0        0        0   
3        0        0        0        0        0        0        0        0   
4        0        0        0        0        0        0        0        0   

   0.39257  
0        0  
1        0  
2        0  

### Pull metadata

In [22]:
# Load the Excel file Metadata
file_path = 'GLORIA_ReadMe_059.xlsx'
df = pd.read_excel(file_path, sheet_name='Sequential region-sector labels')

# Extract the column of interest
labels = df['Sequential_regionSector_labels']

In [32]:
import pandas as pd
import re

# Path to the Excel file
file_path = 'GLORIA_ReadMe_059.xlsx'

# Load the specific sheet and column
df = pd.read_excel(file_path, sheet_name='Sequential region-sector labels', usecols=['Sequential_regionSector_labels'])

# Function to parse the 'Sequential_regionSector_labels' column
def parse_labels(label):
    # Find the last set of parentheses for the country code
    match = re.search(r'\((\w{3})\)', label)
    
    if match:
        # Extract the country code
        country_code = match.group(1).strip()
        
        # Remove the country code part and trim the remaining string
        label_without_code = label[:match.start()].strip()
        industry_part = label[match.end():].strip()
        
        # Extract the country as everything before the last set of parentheses
        last_open_bracket_index = label_without_code.rfind('(')
        if last_open_bracket_index != -1:
            country = label_without_code[:last_open_bracket_index].strip()
        else:
            country = label_without_code.strip()
        
        # The remaining part after the country code is the industry
        industry = industry_part.strip()
        
        return pd.Series([country, country_code, industry], index=['Country', 'Country_Code', 'Industry'])
    else:
        # Handle cases where no country code is found
        return pd.Series([label, None, None], index=['Country', 'Country_Code', 'Industry'])

# Apply the parsing function to the DataFrame
df[['Country', 'Country_Code', 'Industry']] = df['Sequential_regionSector_labels'].apply(parse_labels)

# Drop the original column
# df.drop(columns=['Sequential_regionSector_labels'], inplace=True)

# Print the first few rows of the resulting DataFrame
print(df.head())



                      Sequential_regionSector_labels           Country  \
0      Rest of Americas (XAM) Growing wheat industry  Rest of Americas   
1      Rest of Americas (XAM) Growing maize industry  Rest of Americas   
2  Rest of Americas (XAM) Growing cereals n.e.c i...  Rest of Americas   
3  Rest of Americas (XAM) Growing leguminous crop...  Rest of Americas   
4       Rest of Americas (XAM) Growing rice industry  Rest of Americas   

  Country_Code                                         Industry  
0          XAM                           Growing wheat industry  
1          XAM                           Growing maize industry  
2          XAM                   Growing cereals n.e.c industry  
3          XAM  Growing leguminous crops and oil seeds industry  
4          XAM                            Growing rice industry  


In [29]:

# Print the total number of distinct 'Country' values
num_distinct_countries = df['Country'].nunique()
print(f"\nTotal number of distinct 'Country' values: {num_distinct_countries}")

# Print the total number of distinct 'Country' values
num_distinct_countries = df['Country_Code'].nunique()
print(f"\nTotal number of distinct 'Country_Code' values: {num_distinct_countries}")

# Print the total number of distinct 'Industry' values
num_distinct_industries = df['Industry'].nunique()
print(f"Total number of distinct 'Industry' values: {num_distinct_industries}")


Total number of distinct 'Country' values: 164

Total number of distinct 'Country_Code' values: 164
Total number of distinct 'Industry' values: 240


In [34]:
print(df.shape)

(39360, 4)


In [27]:
df.to_csv('processed_labels.csv', index=False)

### Pull all files for row 5944, 240 industries * 164 countries + 2(year, filename) = 39362 columns
*Accompanying satellite accounts (extensions)*, covering GHG emissions, materials, 
energy, air pollution, land use, water use, biodiversity, skills and employment. 
Files: 
YYYYMMDD_120secMother_AllCountries_002_TQ-Results_YYYY_059_ 
Markup00V(full).csv and YYYYMMDD_120secMother_AllCountries_002_YQ-Results_ 
YYYY_059_Markup00V(full).csv, with TQ being the intermediate satellite block 
(matching intermediate transactions T), and YQ the final satellite block (matching final 
demand Y).

20231117_120secMother_AllCountries_002_TQ-Results_1990_059_Markup001(full)


GLORIA Multi-region input-output (MRIO) database
Files: 
YYYYMMDD_120secMother_AllCountries_002_VAR-Results_YYYY_059_ 
Markup00M(full).csv, with VAR = {T,Y,V} being supply-use transactions (T), final 
demand (Y) and value added (v), and M = 1,…,5 being valuations.  
Note I: Transaction and satellite tables are sized 164 ´ (120´2) = 39,360 region-sector pairs, because of 
the supply-use format. For all labels and the MR-SUT structure, please consult the documentation file 
GLORIA_ReadMe.xlsx.

In [30]:
import os
import zipfile
import pandas as pd
import re

# Path to the folder containing zip files
folder_path = 'Satellite data'

# List to store DataFrames
dataframes = []

def extract_year(filename):
    # Regular expression pattern to match the year in the filename
    pattern = r'_TQ-Results_(\d{4})_'
    match = re.search(pattern, filename)
    if match:
        return match.group(1)
    return None

# Iterate through each zip file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.zip'):
        zip_path = os.path.join(folder_path, filename)
        
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # Iterate through each file in the zip
            for file_info in zip_ref.infolist():
                if 'TQ-Results' in file_info.filename:
                    # Extract the year from the filename
                    year = extract_year(file_info.filename)
                    
                    with zip_ref.open(file_info.filename) as file:
                        # Read the CSV file into a DataFrame, extracting only row 5944
                        df = pd.read_csv(file, skiprows=range(5943), nrows=1, header=None)
                        df.index = [5943]  # Set index to the row number
                        df['Year'] = year  # Add the year column
                        df['Source_Filename'] = file_info.filename  # Add the filename column
                        
                        # Append the DataFrame to the list
                        dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(dataframes)

# Reset index for the final DataFrame (optional)
final_df.reset_index(drop=True, inplace=True)

print(f"Combined DataFrame shape: {final_df.shape}")


Combined DataFrame shape: (39, 39362)


In [ ]:

# Optionally save the final DataFrame to a file
final_df.to_csv('combined_row_5944.csv', index=False)

print(f"Combined DataFrame shape: {final_df.shape}")


In [12]:

# Load the combined DataFrame (replace with your actual loading code)
combined_df = pd.read_csv('combined_row_5944.csv')  # Or use pd.HDFStore if applicable

In [23]:
# Iterate through each zip file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.zip'):
        zip_path = os.path.join(folder_path, filename)
        
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # Iterate through each file in the zip
            for file_info in zip_ref.infolist():
                # Check if the file is a TQ file
                if 'TQ-Results' in file_info.filename:
                    # Read the CSV file into a DataFrame
                    with zip_ref.open(file_info.filename) as file:
                        df = pd.read_csv(file)
                        # Store the DataFrame in the dictionary with a meaningful key
                        dataframes[file_info.filename] = df

MemoryError: Unable to allocate 887. MiB for an array with shape (19437, 5981) and data type float64